In [1]:
from __future__ import division
from __future__ import print_function
import os.path
import numpy as np
import tensorflow as tf
import pickle

In [2]:
base_dir = '/media/nikhil/Data/Experiments/ECG Problem/'

In [3]:
def read_data(category):
    content = []

    with open(base_dir+'sample rr interval data/line_count_rr_with_category.txt', 'r') as fr:
        rr_name = fr.readlines()[0].split('\r')[:-2]
        for names in rr_name:
            name = names.split(' ')
            if name[2] ==category:
                with open(base_dir+'sample rr interval data/'+name[1], 'r') as rr:
                    l = rr.readlines()
                    #for i, x in enumerate(l):
                    l = [x.replace('\tN\n', '') for x in l]
                    #print l
                    content.append(l)
    return content

#Saving data in normal and abnormal since it will be 
#used again and again instead of calling function again and again
normal = read_data("Normal")
abnormal = read_data("Abnormal")

In [4]:
def generate_data(arr, length, window_size):
    #generate multiple array of fixed length from a list
    #taking window size of 10, can make it variable also
    result = []
    for i in xrange(0, len(arr), window_size):
        #print (i)
        if (i+length)>=len(arr):
            break
        result.append(arr[i:i+length])
    #for res in result:
    #    print (len(res))
    return result

In [5]:
#This function calls generate data to agument data and store
#list of lists in new_content

def prepare_data(content,length, window_size):
    new_content = []
    #np.empty([len(content),20])
    j = 0
    for i, con in enumerate(content):
        if len(con) > (length-1):
            result = generate_data(con, length, window_size)
            for res in result:
                new_content.append(res)
                #j = j+1
    return new_content

In [6]:
def abnormalDataList(input_dim, window_size):
    list_test_abnormal = []
    test_abnormal = abnormal
    #print("len1", len(test_abnormal))
    for test_ab in test_abnormal:
        if(len(test_ab)>input_dim):
            ab_list = []
            ab_list.extend(generate_data(test_ab, input_dim, window_size))
            #print (len(test_ab[-input_dim[input_dim_index]-1:-1]))
            ab_list.append(test_ab[-input_dim-1:-1])
            #print ("Hello 2", len(ab_list))
            list_test_abnormal.append(ab_list)
    return list_test_abnormal


def normalDataList(input_dim, window_size):
    content = normal
    new_content = prepare_data(content, input_dim, window_size)
    normal_data_length = len(new_content)
    train_test_split = 0.85
    test_normal = new_content[int(train_test_split*normal_data_length):-1]
    new_content = new_content[:int(train_test_split*normal_data_length)]
    
    return new_content, test_normal

In [7]:
def normalAcc(thresh):
    correct = 0
    incorrect = 0
    for res in norm_res:
        if(res>=thresh):
            incorrect =incorrect+1
        else:
            correct =correct+1
    acc = correct/(correct+incorrect)
    return correct, incorrect, acc
#normalAcc()

def abnormalAcc1(thresh):
    correct = 0
    incorrect = 0
    flag = 0
    x = 0
    y = 0
    for abnorm_res in list_abnorm_res:
        for res in abnorm_res:
            if(res>=thresh):
                correct =correct+1
                flag=1
                break
        if (flag==0):
            incorrect =incorrect+1
        flag=0
    acc = correct/(correct+incorrect)        
    return correct, incorrect, acc
    
#abnormalAcc1()

In [8]:
import random
random.seed(10)

#This function is used in providing batch to tensorflow
#model defined below. It return RR data of size batch size

def next_batch(batch_size):
    i = random.randint(0,len(new_content)-batch_size-1)
    return new_content[i:i+batch_size]

In [9]:
#helper functiont to initialize weights
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.001)
  return tf.Variable(initial)

#helper fucntion to initialize bias
def bias_variable(shape):
  initial = tf.constant(0., shape=shape)
  return tf.Variable(initial)

In [10]:
def tf_graph(input_dim):
    tf.reset_default_graph()
    
    x = tf.placeholder("float", shape=[None, input_dim])
    #print ("shape: ", x.get_shape())
    
    l2_loss = tf.constant(0.0)
    
    #Weight and Bais of input encoder
    W_encoder_input_hidden = weight_variable(
        [input_dim,hidden_encoder_dim])
    b_encoder_input_hidden = bias_variable([hidden_encoder_dim])
    
    l2_loss += tf.nn.l2_loss(W_encoder_input_hidden)
    
    # Hidden layer encoder
    hidden_encoder = tf.nn.relu(tf.matmul(
            x, W_encoder_input_hidden) + b_encoder_input_hidden)
    
    #Weight and Bais of hidden mu
    W_encoder_hidden_mu = weight_variable(
        [hidden_encoder_dim,latent_dim])
    b_encoder_hidden_mu = bias_variable([latent_dim])
    
    l2_loss += tf.nn.l2_loss(W_encoder_hidden_mu)
    
    # Mu encoder
    mu_encoder = tf.matmul(
        hidden_encoder, W_encoder_hidden_mu) + b_encoder_hidden_mu
    
    #Weight and Bais of hidden logvar
    W_encoder_hidden_logvar = weight_variable(
        [hidden_encoder_dim,latent_dim])
    b_encoder_hidden_logvar = bias_variable([latent_dim])
    
    l2_loss += tf.nn.l2_loss(W_encoder_hidden_logvar)
    
    # Sigma encoder
    logvar_encoder = tf.matmul(
        hidden_encoder, W_encoder_hidden_logvar) + b_encoder_hidden_logvar
    
    # Sample epsilon using logvar_encoder
    epsilon = tf.random_normal(tf.shape(logvar_encoder), name='epsilon')
    
    # Sample latent variable
    std_encoder = tf.exp(0.5 * logvar_encoder)
    z = mu_encoder + tf.multiply(std_encoder, epsilon)
    
    #Weight and Bais of z hidden
    W_decoder_z_hidden = weight_variable([latent_dim,hidden_decoder_dim])
    b_decoder_z_hidden = bias_variable([hidden_decoder_dim])
    
    l2_loss += tf.nn.l2_loss(W_decoder_z_hidden)
    
    # Hidden layer decoder
    hidden_decoder = tf.nn.relu(tf.matmul(
            z, W_decoder_z_hidden) + b_decoder_z_hidden)
    
    #Weight and Bais of reconstruction
    W_decoder_hidden_reconstruction = weight_variable(
        [hidden_decoder_dim, input_dim])
    b_decoder_hidden_reconstruction = bias_variable([input_dim])
    
    l2_loss += tf.nn.l2_loss(W_decoder_hidden_reconstruction)
    
    #KL Divergence
    KLD = -0.5 * tf.reduce_sum(1 +
                               logvar_encoder - 
                               tf.pow(mu_encoder, 2)-
                               tf.exp(logvar_encoder),
                               reduction_indices=1)

    x_hat = tf.matmul(
        hidden_decoder, 
        W_decoder_hidden_reconstruction) + b_decoder_hidden_reconstruction

    #BCE loss
    BCE = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(
            logits=x_hat, labels=x), reduction_indices=1)

    loss = tf.reduce_mean(BCE + KLD)

    regularized_loss = loss + lam * l2_loss

    loss_summ = tf.summary.scalar("lowerbound", loss)
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(regularized_loss)
    
    # add op for merging summary
    summary_op = tf.summary.merge_all()

    # add Saver ops
    saver = tf.train.Saver()
    
    with tf.Session() as sess:
        summary_writer = tf.summary.FileWriter('experiment',
                                               graph=sess.graph)
        if os.path.isfile(base_dir+'VAE-TensorFlow-master/save/model.ckpt'):
            print("Restoring saved parameters")
            saver.restore(
                sess, base_dir+'VAE-TensorFlow-mastersave/model.ckpt')
        else:
            print("Initializing parameters")
            sess.run(tf.global_variables_initializer())

        for step in range(1, n_steps):
            batch = next_batch(batch_size)

            feed_dict = {x: batch}
            _, cur_loss, summary_str = sess.run(
                [train_step, regularized_loss, summary_op], feed_dict=feed_dict)
            summary_writer.add_summary(summary_str, step)
            
            if step ==1:
                max_threshold = min_threshold = cur_loss

            if step % 500 == 0:
                save_path = saver.save(
                    sess, base_dir+"VAE-TensorFlow-master/save/model.ckpt")
                #print("Step {0} | Loss: {1}".format(step, cur_loss))
                
                if max_threshold < cur_loss:
                    max_threshold = cur_loss
                if min_threshold > cur_loss:
                    min_threshold = cur_loss
                
            
                
        #start_threshold = cur_loss
        norm_res = []
        #abnorm_res = []
        list_abnorm_res = []
        for dat in test_normal:
            norm_res.append(sess.run(regularized_loss, feed_dict={x:[dat]}))
        #for dat in test_abnormal:
        #    abnorm_res.append(sess.run(loss, feed_dict={x:[dat]}))
        for test_abnormal in list_test_abnormal:
            abnorm_res = []
            for dat in test_abnormal:
                #print (len(dat))
                abnorm_res.append(sess.run(regularized_loss, feed_dict={x:[dat]}))
            list_abnorm_res.append(abnorm_res)
            
    return norm_res, list_abnorm_res, min_threshold, max_threshold

In [ ]:
def result(thresh, window_size, input_dim):
    _, _, acc1 = normalAcc(thresh)
    _, _, acc2 = abnormalAcc1(thresh)
    '''
    print("Window| index_len|Thresh| Normal Acc | Abnormal Acc |  Score")
    print(window_size, "       ",input_dim, "    ", "{0:.2f}".format(thresh), "  ",
          "{0:.5f}".format(acc1)," ","{0:.5f}".format(acc2)," ",
          "{0:.5f}".format(((acc1+acc2)/2)))
          
    '''
    acc3 = ((acc1+acc2)/2)
    
    return acc1, acc2, acc3
    
                    
                
'''
    print("Window Size ", window_size)
    print("index_length ", input_dim)
    print("Threshold", threshold)
    print("Normal Acc")
    
    print(acc1)
    print("Abnormal Acc")
    print(acc2)
    print((acc1+acc2)/2)
    
    '''

'\n    print("Window Size ", window_size)\n    print("index_length ", input_dim)\n    print("Threshold", threshold)\n    print("Normal Acc")\n    \n    print(acc1)\n    print("Abnormal Acc")\n    print(acc2)\n    print((acc1+acc2)/2)\n    \n    '

In [ ]:
n_steps = int(1e4)
batch_size = 100

#Hyperparameters 
grid_hidden_encoder_dim = [50, 75, 100, 150, 200]
grid_hidden_decoder_dim = [50, 75, 100, 150, 200]
grid_latent_dim = [5, 10,15, 20]
grid_lam = [0.01, 0.02, 0.03, 0.04, 0.05, 0.07, 0.1]
grid_learning_rate = [0.00001,0.00003, 0.0001, 0.0003]
grid_window_size = [5, 10, 15,20]
grid_input_dim = [10, 20, 30, 40, 50, 60, 70]


#hidden_encoder_dim = 50
#hidden_decoder_dim = 50
#latent_dim = 10
#lam = 0.02
#learning_rate = 0.00001

#preparing data of length of interval 20,
#This function will agument that RR dataset by creating dataset of 
#fixed length using a window size of 10
#window_size = 10

#index of internal interval(input_dim)
#input_dim_index = 2
#input_dim = [20, 30, 50, 70]
#threshold = 29.3


index = ['grid_input_dim', 'grid_window_size', 'grid_hidden_encoder_dim', 
         'grid_hidden_decoder_dim', 'grid_latent_dim', 'grid_lam', 
         'grid_learning_rate', 'Threshold', 'Normal Accuracy',
         'Abnormal Accuracy', 'Score']
results = []


for input_dim in grid_input_dim:
    for window_size in grid_window_size:
        for hidden_encoder_dim in grid_hidden_encoder_dim:
            hidden_decoder_dim = hidden_encoder_dim
            for latent_dim in grid_latent_dim:
                for lam in grid_lam:
                    for learning_rate in grid_learning_rate:
                        new_content, test_normal = normalDataList(input_dim, window_size)
                        #print ("normal train len ", len(new_content))
                        #print("normal test len", len(test_normal))

                        list_test_abnormal = abnormalDataList(input_dim, window_size)
                        #print ("abnormal test len", len(list_test_abnormal))

                        norm_res, list_abnorm_res, start_threshold, end_threshold = tf_graph(input_dim)
                        #print("Start Threshold ", start_threshold, "end_threshold ", end_threshold)
                        
                        #end_threshold = float(end_threshold)
                        for thresh in np.arange(start_threshold, end_threshold, 0.2):
                            acc1, acc2, acc3 = result(thresh, window_size, input_dim)
                            results.append([input_dim, window_size, hidden_encoder_dim, hidden_decoder_dim, 
                                            latent_dim, lam, learning_rate, thresh, acc1, acc2, acc3])
                            #print (results)
                            
                            
                            



Initializing parameters


In [ ]:
#print(len(abnorm_res))
print(len(norm_res))
print(len(list_abnorm_res))

In [ ]:
with open('/media/nikhil/Data/Experiments/ECG Problem/Code/result/outfile', 'wb') as fp:
    pickle.dump(results, fp)
    
    
with open ('/media/nikhil/Data/Experiments/ECG Problem/Code/result/outfile', 'rb') as fp:
    results = pickle.load(fp)

